A decorater essentially a funciton that takes another funciton as an argument and extends it behaviour. It is like wrapping paper around a gift - it adds functionality without changing the original item inside. 

In [35]:

from typing import List

In [25]:
# basic Decorator Structre: 

def my_decorator(func):
    def my_wrapper():
        print("let's do something before the function is called!")
        func()
        print("let's do something AFTER the function has been called!!")
    return my_wrapper

In [26]:
@my_decorator
def say_hello():
    print("Hello")

In [27]:
say_hello()

let's do something before the function is called!
Hello
let's do something AFTER the function has been called!!


In [39]:
## Use @wraps decorator inside your decorator itself!!! by why!!

@my_decorator
def say_hello_again():
    """ this func greets you with a hello!"""
    print("Hello")

print(say_hello_again.__name__)
print(say_hello_again.__doc__)

## see, this is how you loose info about the fun name and docstrings if you just use the decorator, using @wraps prevents that!!

my_wrapper
None


In [41]:
from functools import wraps

def my_better_decorator(func):
    @wraps(func)
    def my_wrapper():
        print("let's do something before the function is called!")
        func()
        print("let's do something AFTER the function has been called!!")
    return my_wrapper

@my_better_decorator
def say_hello_again():
    """this func greets you with a hello!"""
    print("Hello")

print(say_hello_again.__name__)
print(say_hello_again.__doc__)

say_hello_again
this func greets you with a hello!


In [23]:
## Timining Funciton Execution

import time

def measure_time(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.2f} seconds to execute!!")
        
        # this needs to return the result, otherwise it will just execute it, and not return anything. 
        return result 
    
    return wrapper

@measure_time
def calculate_squares(n):
    # let's force the a quick nap as soon as you get some work!!
    time.sleep(3)
    return [i**2 for i in range(n)]

In [24]:
calculate_squares(10)

calculate_squares took 3.00 seconds to execute!!


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [28]:
## Practice : build a decorator that measure memory usage

In [29]:
import sys

my_list = [1,2,3,4,5]

sys.getsizeof(my_list)

96

In [38]:
# simple decorator which just calculates the memory used by the result of the function, not the entire operation within the func
def get_memory_size(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        memory_used = sys.getsizeof(result)
        print(f"memory used by this {func.__name__} = {memory_used}")
        return result 
    return wrapper


In [31]:
@get_memory_size
def any_ranodm_func(n): 
    return {i: str(i**1/2) for i in range(10)}


In [32]:
any_ranodm_func(10)

memory used by this any_ranodm_func = 360


{0: '0.0',
 1: '0.5',
 2: '1.0',
 3: '1.5',
 4: '2.0',
 5: '2.5',
 6: '3.0',
 7: '3.5',
 8: '4.0',
 9: '4.5'}

In [ ]:
## decorator which checks the all t

In [36]:
@get_memory_size
def create_large_list(size: int) -> List[int]:
    """Creates a large list of integers"""
    return list(range(size))

In [37]:
create_large_list(100)

memory used by this create_large_list = 856


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [55]:
# better way to check the memory usage 

import tracemalloc

tracemalloc.start()

start_memory, peak = tracemalloc.get_traced_memory()
some_list = [i for i in range(100)]
current, peak = tracemalloc.get_traced_memory()

tracemalloc.stop()

print(f"memory usage : {current - start_memory}")

memory usage : 920


In [56]:
# making a decorator which tracks memory use of a func 

def measure_memory(func):
    # standard practice
    wraps(func)

    def memory_wrapper(*args, **kwargs):
        # start memory tracking 
        tracemalloc.start()
        # get the current mem usage 
        start_memory, peak = tracemalloc.get_traced_memory()
        # run the func
        result = func(*args, **kwargs)
        # memore usage after the func executes 
        current, peak = tracemalloc.get_traced_memory()
        # stop tracking
        tracemalloc.stop()
        # memory usage
        memory_diff = current - start_memory
        print(f"\nFunction: {func.__name__}")
        print(f"Current memory usage: {current / 10**6:.2f} MB")
        print(f"Peak memory usage: {peak / 10**6:.2f} MB")
        print(f"Memory difference: {memory_diff / 10**6:.2f} MB")

        return result
    
    return memory_wrapper



In [57]:
def create_large_list(size: int) -> List[int]:
    """Creates a large list of integers"""
    return list(range(size))

def string_concatenation(n: int) -> str:
    """Performs repeated string concatenation"""
    result = ""
    for i in range(n):
        result += f"string{i}"
    return result

@measure_memory
def test_memory_usage():
    large_list = create_large_list(1000000)
    string = string_concatenation(10000)
    return "operations completed"

In [58]:
test_memory_usage()


Function: test_memory_usage
Current memory usage: 0.00 MB
Peak memory usage: 36.09 MB
Memory difference: 0.00 MB


'operations completed'